In [5]:
import rospy
import numpy as np
from cookiecutter.main import cookiecutter
import collections
from pathlib import Path
import json

In [6]:
def get_param(name, default):
    try:
        value = rospy.get_param(name, default)
    except ConnectionRefusedError:
        value = default
    print('param "{}" = "{}"'.format(name, value))
    return value

SEED = int(get_param('~seed', '178'))
WORLD_NAME = get_param('~world_name', 'tomato_field')
MODEL_NAME_PREFIX = get_param('~model_name_prefix', 'tomato')
OUT_PATH = Path(get_param('~out_path', Path.cwd() / '../generated/test01')).resolve()
MODEL_TEMPLATE = Path(get_param('~model_template', Path.cwd() / '../templates/tomato_model')).resolve()
WORLD_TEMPLATE = Path(get_param('~world_template', Path.cwd() / '../templates/tomato_world')).resolve()
ROW_COUNT = int(get_param('~row_count', '2'))
ROW_LENGTH = int(get_param('~row_length', '3'))
ROW_DIST = float(get_param('~row_dist', '2.0'))
CROP_DIST = float(get_param('~crop_dist', '1.4'))

np.random.seed(SEED)


param "~seed" = "178"
param "~world_name" = "tomato_field"
param "~model_name_prefix" = "tomato"
param "~out_path" = "/home/azazdeaz/repos/fields_ignition/src/packs/fields_ignition/scripts/../generated/test01"
param "~model_template" = "/home/azazdeaz/repos/fields_ignition/src/packs/fields_ignition/scripts/../templates/tomato_model"
param "~world_template" = "/home/azazdeaz/repos/fields_ignition/src/packs/fields_ignition/scripts/../templates/tomato_world"
param "~row_count" = "2"
param "~row_length" = "3"
param "~row_dist" = "2.0"
param "~crop_dist" = "1.4"


In [7]:
class Markers:
    markers = []
    last_id = 0

    @staticmethod
    def next_id():
        Markers.last_id += 1
        return Markers.last_id

    @staticmethod
    def reset():
        Markers.markers = []
    
    @staticmethod
    def add_plant(x, y, z):
        id = Markers.next_id()
        Markers.markers.append({
            'marker_type': 'PLANT',
            'id': id,
            'translation': [x, y, z]
        })
        return id
    

    @staticmethod
    def add_fruit(x, y, z, plant_id):
        id = Markers.next_id()
        Markers.markers.append({
            'marker_type': 'FRUIT',
            'id': id,
            'translation': [x, y, z],
            'plant_id': plant_id
        })
        return id
    
    @staticmethod
    def dumps():
        return json.dumps(Markers.markers, indent=4)

In [8]:
models = {'list': []}
Markers.reset()

for x in range(ROW_COUNT):
    for y in range(ROW_LENGTH):
        model_name = 'tomato_{}'.format(x * ROW_LENGTH + y)

        cookiecutter(str(MODEL_TEMPLATE),
             output_dir=str(OUT_PATH), 
             overwrite_if_exists=True, 
             no_input=True,
             extra_context={'world_name': WORLD_NAME, 'model_name': model_name})

        x_pos, y_pos, z_pos = x * ROW_DIST, y * CROP_DIST, 0
        models['list'].append({
            'model': model_name,
            'name': model_name,
            'pose': '{} {} 0 0 0 0'.format(x_pos, y_pos)
        })
        seed = np.random.randint(10000)
        dir = (OUT_PATH / WORLD_NAME / model_name).resolve()
        dir_blender = (Path.cwd() / '../blender').resolve()
        blend = str(dir_blender / 'tomato_gen.blend')
        script = str(dir_blender / 'tomato_gen.py')
        ! blender $blend --background --python $script -- --model_dir $dir --seed $seed

        plant_id = Markers.add_plant(x_pos, y_pos, z_pos)
        with open(dir / 'markers.json') as markers_file:
            plant_markers = json.load(markers_file)
            for marker in plant_markers:
                if marker['marker_type'] == 'FRUIT':
                    Markers.add_fruit(
                        marker['translation'][0] + x_pos,
                        marker['translation'][1] + y_pos,
                        marker['translation'][2] + z_pos,
                        plant_id
                    )
            

cookiecutter(str(WORLD_TEMPLATE),
             output_dir=str(OUT_PATH), 
             overwrite_if_exists=True, 
             no_input=True,
             extra_context={'world_name': WORLD_NAME, 'models': models})

with open(OUT_PATH / WORLD_NAME / 'markers.json', 'w') as outfile:
    json.dump(Markers.markers, outfile, indent=4, sort_keys=True)

pose_fruit.192'], bpy.data.objects['pose_fruit.193'], bpy.data.objects['pose_fruit.194'], bpy.data.objects['pose_fruit.195'], bpy.data.objects['pose_fruit.196'], bpy.data.objects['pose_fruit.197'], bpy.data.objects['pose_fruit.198'], bpy.data.objects['pose_fruit.199'], bpy.data.objects['pose_fruit.200'], bpy.data.objects['pose_fruit.201'], bpy.data.objects['pose_fruit.202'], bpy.data.objects['pose_fruit.203'], bpy.data.objects['pose_fruit.204'], bpy.data.objects['pose_fruit.205'], bpy.data.objects['pose_fruit.206']]
[bpy.data.objects['pose_fruit.181'], bpy.data.objects['pose_fruit.182'], bpy.data.objects['pose_fruit.183'], bpy.data.objects['pose_fruit.184'], bpy.data.objects['pose_fruit.185'], bpy.data.objects['pose_fruit.186'], bpy.data.objects['pose_fruit.187'], bpy.data.objects['pose_fruit.188'], bpy.data.objects['pose_fruit.189'], bpy.data.objects['pose_fruit.190'], bpy.data.objects['pose_fruit.191'], bpy.data.objects['pose_fruit.192'], bpy.data.objects['pose_fruit.193'], bpy.data.